## Read and Save Query Status in CSV for all Workflows

<br>

**Read all the JSON files for all the workflows and print out the messages and query status to a CSV file**

<br>

In [15]:
# import all the modules. NB: submit_run_ars_modules contains all the modules to submit job to ARAX

import json
import requests
from gamma_viewer import GammaViewer
from IPython.display import display
#from submit_run_ars_modules import submit_to_ars, submit_to_devars, printjson, retrieve_devars_results
import glob 
import os
from collections import defaultdict
import pandas as pd
from time import sleep
from os import path
from datetime import datetime
import numpy as np



In [16]:
def flatten_list(_2d_list):
    flat_list = []
    # Iterate through the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

In [17]:
def submit_to_ars(m,ars_url='https://ars.transltr.io/ars/api',arax_url='https://arax.ncats.io'):
    submit_url=f'{ars_url}/submit'
    response = requests.post(submit_url,json=m)
    try:
        message_id = response.json()['pk']
    except:
        print('fail')
        message_id = None
    print(f'{arax_url}/?source=ARS&id={message_id}')
    return message_id

##https://ars.ci.transltr.io/ars/api

def retrieve_ars_results(mid, name, check_sheet, ars_url='https://ars.transltr.io/ars/api'):
    pk = 'https://arax.ncats.io/?source=ARS&id=' + mid
    message_url = f'{ars_url}/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    print( j['status'] )
    results = {}
    dictionary = {}
    dictionary_2 = {}
    dict3 = {}
    for child in j['children']:
        print(child['status'])
        error_code = child['code']
        
        if child['status']  == 'Done':
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
                    
                    if name in list(check_sheet.Workflow):
                        print(name)
                        dfy = check_sheet[check_sheet['Workflow']== name]

                        dfy.reset_index(drop=True)

                        for index,curie_id in enumerate(dfy.Curie):
                            print(index,curie_id)
                            node_num = dfy.iloc[index][3]
                            query_id = curie_id
                            if np.isnan(dfy.iloc[index][2]) == True:
                                len_check = len(child_response['fields']['data']['message']['results'])
                                len_check = int(len_check)
                            else:
                                len_check = int(dfy.iloc[index][2])

                            #print(node_num, query_id, len_check)

                            locs = []
                            for x, val in enumerate(child_response['fields']['data']['message']['results']):
                                #print(val)

                                if x < len_check:

                                    if query_id in val['node_bindings'][node_num][0]['id']:
                                        locs.append(x)
                            if not locs:
                                check_result = f'False'
                                print(check_result)
                                #pass
                            else:
                                check_result = f'True'
                                print('curie id:', query_id, ': INCLUDED at postion N ==', locs, 'on', node_num)

                            dict3[curie_id] = check_result    

                if child_response['fields']['data']['message']['knowledge_graph']['edges']:
                    if child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
                            edge_ex = child_response['fields']['data']['message']['knowledge_graph']['edges']
                            test_att_values =[]
                            for val in child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
                                #print(val)
                                
                                for tx in edge_ex[val]['attributes']:
                                    if (tx['attribute_type_id'] == 'biolink:primary_knowledge_source') or (tx['attribute_type_id'] == 'biolink:original_knowledge_source') or (tx['attribute_type_id'] == 'biolink:aggregator_knowledge_source') :
                                        
                                        
                                        value_att = tx['value']
                        
                                        test_att_values.append(value_att)
                                        test_att = set(flatten_list(test_att_values))
                                        
                                        
                                        dictionary_2[child['actor']['agent']] = test_att
                    #else:
                        #dictionary_2[child['actor']['agent']] = [] 
                #else:
                   # dictionary_2[child['actor']['agent']] = []
            
            except Exception as e:
                nresults=0
                child['status'] = 'ARS Error'
                #dictionary_2[child['actor']['agent']] = []
                
            
        
        elif child['status'] == 'Error':
            nresults=0
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
                #dictionary_2[child['actor']['agent']] = []
            except Exception as e:
                #print(e)
                child['status'] = 'ARS Error'
                #dictionary_2[child['actor']['agent']] = []
        
        
        else:
            nresults = 0
            #dictionary_2[child['actor']['agent']] = []
            
        dictionary['pk_id'] =  pk  
            
        if ((child['status'] == 'Done') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'No Results' ': ' + str(error_code)
            #test =  [child['actor']['agent'], 'No Results']
        elif ((child['status'] == 'ARS Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'ARS Error' ': ' + str(error_code)
        elif ((child['status'] == 'Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'Error' ': ' + str(error_code)
            #test =  [child['actor']['agent'], 'ARS Error']
        elif ((child['status'] == 'Done') & (nresults != 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Results' ': ' + str(error_code) + ' ' + str(dict3)
        elif ((child['status'] == 'Unknown') & (nresults == 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Unknown' ': ' + str(error_code)
        
        
        print(child['actor']['agent'], child['status'], nresults)
        #test =  [child['actor']['agent'], child['status'], nresults]
        #test2.append(test)
    return [dictionary, dictionary_2]


#def submit_to_devars(m):
#    return submit_to_ars(m,ars_url='https://ars-dev.transltr.io/ars/api',arax_url='https://arax.ncats.io')

#def retrieve_devars_results(m):
#     return retrieve_ars_results(m,ars_url='https://ars-dev.transltr.io/ars/api')

def printjson(j):
    print(json.dumps(j,indent=4))
    
def make_hyperlink(value):
    return '=HYPERLINK("%s", "%s")' % (value.format(value), value)

In [18]:
check_sheet = pd.read_csv("/Users/priyash/Documents/GitHub/minihackathons/Notebooks/Query results to include or exclude - Sheet1.csv", header=0)

In [19]:
check_sheet

,Workflow,Curie,N (size of list of results),Query node ID,Include/Exclude,Unnamed: 5,Unnamed: 6
0,A.0_RHOBTB2_direct.json,PUBCHEM.COMPOUND:2950270,10.0,n1,Include,NaN,Note: this example row means that for workflow...
1,B.1_DILI-three-hop-interacts-with.json,CHEBI:41879,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
2,B.1_DILI-three-hop-interacts-with.json,MESH:D000077185,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
3,B.1_DILI-three-hop-interacts-with.json,MESH:D000077385,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
4,B.1_DILI-three-hop-interacts-with.json,MESH:D003474,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
5,B.1_DILI-three-hop-interacts-with.json,RXNORM:40068,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
6,B.1_DILI-three-hop-interacts-with.json,PUBCHEM.COMPOUND:5877,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
7,B.1_DILI-three-hop-interacts-with.json,PUBCHEM.COMPOUND:5755,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
8,C.1a_SmallMolecule_real_world_evidence_MultScl...,MONDO:0005301,100.0,n00,include,NaN,NaN
9,C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallM...,CHEBI:45783,100.0,n3,include,NaN,NaN


<br>

**The below code reads each JSON files from the Workflows A through D (subdirectories). The queries are submitted to ARAX and output is saved in a dictionary, where the key is the file name of the JSON to denote which query is being run and the values assigned to the key is the query id**

<br>


In [20]:
PATH = r'/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo'
EXT = "*.json"
dict_workflows = {}
for root, dirs, files in os.walk(PATH): # step 1: accessing file
    #print(root)
    for name in files:
        
        if name.endswith((".json")):
            if (name == 'C.1_Template_SmallMolecule_real_world_evidence_Disease.json') or (name == 'C.2_Template_Drug_Disease_GeneSet_interacts_with_SmallMolecule.json') or (name == 'C.3_Template_Disease_related_to_Drug.json'):
                pass
            else:
                file_read = path.join(root, name)
                dir_name = (os.path.splitext(os.path.basename(root))[0])
                print(file_read)

                filename = (os.path.splitext(os.path.basename(file_read))[0])
                print(filename)
                with open(file_read,'r') as inf:
                    query = json.load(inf)

                    kcresult = submit_to_ars(query)

                    sleep(350)

                    result_status = retrieve_ars_results(kcresult, name, check_sheet)


                    dict_workflows[filename] = kcresult

                    sleep(100)

/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.0_RHOBTB2_direct.json
A.0_RHOBTB2_direct
https://arax.ncats.io/?source=ARS&id=01297837-569a-4b85-ab0c-7ea95cad6937
Done
Done
kp-cam Done 0
Error
ara-aragorn-exp Error 0
Done
ara-unsecret Done 0
Done
kp-textmining Done 0
Error
kp-cohd Error 0
Error
ara-robokop Error 0
Done
ara-aragorn Done 0
Done
ara-explanatory Done 0
Done
ara-bte Done 0
Done
kp-chp Done 0
Done
kp-openpredict Done 0
Done
kp-icees-dili Done 0
Done
kp-icees Done 0
Done
ara-arax Done 0
Done
kp-molecular Done 0
Done
A.0_RHOBTB2_direct.json
0 PUBCHEM.COMPOUND:2950270
curie id: PUBCHEM.COMPOUND:2950270 : INCLUDED at postion N == [0] on n1
ara-improving Done 1
Done
kp-genetics Done 0
Unknown
ara-ncats Unknown 0
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.0_RHOBTB2_direct_inverse.json
A.0_RHOBTB2_direct_inverse
https://arax.ncats.io/?source=ARS&id=8b3142ae-4e09-4e6d-9990-809d3cbbd587
Done
Done
kp-chp Done 0
Done
kp-icees-dili Do

https://arax.ncats.io/?source=ARS&id=69048de0-a8da-4716-8f96-dc810eb4e9c7
Done
Done
ara-explanatory Done 0
Error
kp-cohd Error 0
Done
ara-improving Done 1000
Error
kp-molecular Error 0
Done
ara-arax Done 500
Done
kp-icees ARS Error 0
Done
kp-openpredict Done 0
Error
ara-robokop Error 0
Done
kp-icees-dili ARS Error 0
Done
ara-aragorn Done 0
Done
kp-textmining Done 1691
Unknown
kp-genetics Unknown 0
Error
ara-aragorn-exp Error 0
Done
ara-unsecret Done 0
Done
kp-cam Done 0
Done
ara-bte Done 1691
Unknown
ara-ncats Unknown 0
Error
kp-chp Error 0
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/future/A.2a_expanded_RHOBTB2_twohop_constrained.json
A.2a_expanded_RHOBTB2_twohop_constrained
https://arax.ncats.io/?source=ARS&id=51991eb6-90c2-40b7-ac4b-bee68ca524e3
Done
Done
kp-icees-dili ARS Error 0
Unknown
kp-genetics Unknown 0
Error
kp-chp Error 0
Done
kp-textmining Done 86
Error
kp-molecular Error 0
Done
ara-improving ARS Error 0
Done
ara-explanatory Done 0
Done
ara-aragor

https://arax.ncats.io/?source=ARS&id=8545ddfd-fffd-4a65-b41a-dd7d9f4cabd3
Running
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowC/C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule.json
C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule
https://arax.ncats.io/?source=ARS&id=21680308-299e-4766-a9f5-57f15e5c54a5
Running


<br>

### Codes below are for recording messages and generating outout as csv

<br>

In [21]:
dict_workflows

{'A.0_RHOBTB2_direct': '90a3ab93-d05c-4692-83c9-1a9de50ef2cc',
 'A.0_RHOBTB2_direct_inverse': '8b3142ae-4e09-4e6d-9990-809d3cbbd587',
 'A.2_RHOBTB2_twohop': '9ad71701-af5c-4914-858b-a961827c2b2d',
 'A.2a_expanded_RHOBTB2_twohop': '566ba27c-7007-403c-bffe-56d9939e1524',
 'A.3_KCNMA1': '6661979d-dbca-429d-898d-c52a2b7530ac',
 'A.8_EGFR_simple': '545a14e8-30ea-4823-8011-7b301630718f',
 'A.Multiomics_BigGIM_DR_KP_RHOBTB2': '57acf77a-37c3-468e-aea8-9cc5455b09a1',
 'A.Multiomics_BigGIM_DR_KP_EGFR': '91a42b9e-ffe0-4e41-bb2f-e21c9f2b2638',
 'A.9_EGFR_advanced': 'f30f7e1e-3584-4f4f-b0a0-5033d13e97b2',
 'A.1_RHOBTB2': 'e9ea07a4-d9d1-467d-8dfe-3d732d2f19cd',
 'A.2a_RHOBTB2_twohop': 'cf8a9ce5-f022-4781-8159-5a5cd8bb9aae',
 'A.2_RHOBTB2_twohop_constrained': '69048de0-a8da-4716-8f96-dc810eb4e9c7',
 'A.2a_expanded_RHOBTB2_twohop_constrained': '51991eb6-90c2-40b7-ac4b-bee68ca524e3',
 'B.2b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol': '80c2673d-e5a9-4ea4-9fea-1afb176fabe3',
 'B.3b_DILI-two-ho

In [22]:
sleep(4000)

In [23]:
workflow_result_messages = {}
for keys, val in dict_workflows.items():
    name = keys + '.json'
    print(name, val)
    

    result_status = retrieve_ars_results(val, name, check_sheet)

    workflow_result_messages[keys] = result_status

A.0_RHOBTB2_direct.json 90a3ab93-d05c-4692-83c9-1a9de50ef2cc
Done
Done
ara-aragorn Done 0
Error
ara-aragorn-exp Error 0
Done
kp-cam Done 0
Error
ara-explanatory Error 0
Done
kp-icees Done 0
Done
A.0_RHOBTB2_direct.json
0 PUBCHEM.COMPOUND:2950270
curie id: PUBCHEM.COMPOUND:2950270 : INCLUDED at postion N == [0] on n1
ara-improving Done 1
Done
ara-arax Done 0
Error
ara-robokop Error 0
Done
kp-openpredict Done 0
Done
kp-icees-dili Done 0
Done
ara-bte Done 0
Done
kp-textmining Done 0
Done
ara-unsecret Done 0
Done
kp-chp Done 0
Error
kp-cohd Error 0
Done
kp-molecular Done 0
Done
kp-genetics Done 0
Unknown
ara-ncats Unknown 0
A.0_RHOBTB2_direct_inverse.json 8b3142ae-4e09-4e6d-9990-809d3cbbd587
Done
Done
kp-chp Done 0
Done
kp-icees-dili Done 0
Unknown
ara-ncats Unknown 0
Done
kp-icees Done 0
Done
kp-openpredict Done 0
Error
ara-robokop Error 0
Done
kp-molecular Done 0
Done
kp-genetics Done 0
Done
ara-aragorn Done 5
Done
ara-arax Done 4
Done
ara-improving ARS Error 0
Done
ara-bte Done 1
Done
a

ara-aragorn Done 0
Done
kp-textmining Done 0
Error
kp-cohd Error 0
Done
kp-chp Done 0
Done
kp-icees-dili ARS Error 0
Done
ara-unsecret Done 0
Error
ara-arax Error 0
Done
kp-cam Done 0
Error
ara-robokop Error 0
Unknown
kp-genetics Unknown 0
Unknown
ara-ncats Unknown 0
Error
ara-bte Error 0
Error
ara-aragorn-exp Error 0
B.2a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone.json d32ac561-aa8a-40f1-8ae2-67cce398bc46
Done
Done
kp-molecular Done 242
Error
ara-aragorn Error 0
Done
kp-icees Done 0
Done
kp-textmining Done 1198
Done
kp-icees-dili Done 0
Done
kp-cohd Done 50
Done
kp-chp Done 0
Done
ara-explanatory Done 3084
Done
kp-openpredict Done 300
Unknown
ara-ncats Unknown 0
Done
ara-arax Done 500
Error
ara-aragorn-exp Error 0
Error
ara-robokop Error 0
Done
ara-improving Done 191
Done
kp-genetics Done 0
Done
ara-unsecret Done 347
Done
ara-bte Done 1198
Done
kp-cam Done 0
B.3a_DILI-two-hop-chemical-entity.json 3f798292-3ac6-42d4-bbfa-b0d55abacedf
Done
Error
kp-molecular Error 0
Done
ara-bte

ara-aragorn Done 424
Done
ara-improving ARS Error 0
Done
kp-genetics Done 0
Error
ara-aragorn-exp Error 0
Done
C.1a_SmallMolecule_real_world_evidence_MultSclerosis.json
0 MONDO:0005301
curie id: MONDO:0005301 : INCLUDED at postion N == [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 63, 64, 66, 67, 69, 70, 72, 73, 74, 75, 76, 86, 94] on n00
ara-arax Done 142
Done
C.1a_SmallMolecule_real_world_evidence_MultSclerosis.json
0 MONDO:0005301
curie id: MONDO:0005301 : INCLUDED at postion N == [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67] on n00
kp-textmining Done 68
Done
C.1a_SmallMolecule_real_world_evi

### Creating dataframe for workflows with PK

<br>

In [24]:
## Convert mesages to a dataframe
col = []
final_dict = defaultdict(list)


for k in sorted(workflow_result_messages):
    print(k)
    
    col.append(k)
    
    
    
    #count = 0
    for key, value in workflow_result_messages[k][0].items():
        
        
        #count= count+1
        final_dict[key].append(value)

    final_dict = dict(final_dict)
    #print(count)
df = pd.DataFrame(final_dict).T
df.rename(columns=dict(zip(df.columns, col)), inplace=True)

A.0_RHOBTB2_direct
A.0_RHOBTB2_direct_inverse
A.1_RHOBTB2
A.2_RHOBTB2_twohop
A.2_RHOBTB2_twohop_constrained
A.2a_RHOBTB2_twohop
A.2a_expanded_RHOBTB2_twohop
A.2a_expanded_RHOBTB2_twohop_constrained
A.3_KCNMA1
A.8_EGFR_simple
A.9_EGFR_advanced
A.Multiomics_BigGIM_DR_KP_EGFR
A.Multiomics_BigGIM_DR_KP_RHOBTB2
B.1_DILI-three-hop
B.2a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone
B.2b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol
B.2c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin
B.2d_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone
B.2e_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5865_Prednisone
B.3a_DILI-two-hop-chemical-entity
B.3b_DILI-two-hop-drug
C.1a_SmallMolecule_real_world_evidence_MultSclerosis
C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule
C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule
C.3a_MultSclerosis_related_to_Nimodipine
D.1_parkinsons-crohns
D.2_ssri-heart-disease
D.3_ssri-heart-disease-one-hop
D.4_tryptophan-kynurenine
D.6_metfo

In [25]:
df

,A.0_RHOBTB2_direct,A.0_RHOBTB2_direct_inverse,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,...,B.3b_DILI-two-hop-drug,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule,C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
pk_id,https://arax.ncats.io/?source=ARS&id=90a3ab93-...,https://arax.ncats.io/?source=ARS&id=8b3142ae-...,https://arax.ncats.io/?source=ARS&id=e9ea07a4-...,https://arax.ncats.io/?source=ARS&id=9ad71701-...,https://arax.ncats.io/?source=ARS&id=69048de0-...,https://arax.ncats.io/?source=ARS&id=cf8a9ce5-...,https://arax.ncats.io/?source=ARS&id=566ba27c-...,https://arax.ncats.io/?source=ARS&id=51991eb6-...,https://arax.ncats.io/?source=ARS&id=6661979d-...,https://arax.ncats.io/?source=ARS&id=545a14e8-...,...,https://arax.ncats.io/?source=ARS&id=8e0c4d64-...,https://arax.ncats.io/?source=ARS&id=439a699a-...,https://arax.ncats.io/?source=ARS&id=21680308-...,https://arax.ncats.io/?source=ARS&id=20dddccd-...,https://arax.ncats.io/?source=ARS&id=8545ddfd-...,https://arax.ncats.io/?source=ARS&id=da789ff3-...,https://arax.ncats.io/?source=ARS&id=87a3de47-...,https://arax.ncats.io/?source=ARS&id=ad6f1db8-...,https://arax.ncats.io/?source=ARS&id=ac9c22cc-...,https://arax.ncats.io/?source=ARS&id=943660a4-...
ara-aragorn,No Results: 200,Results: 200 {},Results: 200 {},Error: 598,No Results: 200,Results: 200 {},Results: 200 {},No Results: 200,Results: 200 {},Results: 200 {},...,Error: 598,Results: 200 {'MONDO:0005301': 'True'},Error: 598,Results: 200 {'CHEMBL.COMPOUND:CHEMBL1201607':...,Results: 200 {},Error: 598,Error: 598,Results: 200 {},Results: 200 {},Error: 598
ara-aragorn-exp,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,...,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404
kp-cam,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-explanatory,Error: 500,Results: 200 {},Results: 200 {},No Results: 200,No Results: 200,Results: 200 {},No Results: 200,No Results: 200,Results: 200 {},No Results: 200,...,No Results: 200,Results: 200 {'MONDO:0005301': 'True'},No Results: 200,No Results: 200,Results: 200 {},Error: 504,ARS Error: 200,Results: 200 {},Results: 200 {},Error: 504
kp-icees,No Results: 200,No Results: 200,No Results: 200,ARS Error: 200,ARS Error: 200,ARS Error: 200,ARS Error: 200,ARS Error: 200,No Results: 200,No Results: 200,...,ARS Error: 200,Error: 598,ARS Error: 200,ARS Error: 200,No Results: 200,Error: 598,ARS Error: 200,Error: 598,ARS Error: 200,Error: 598
ara-improving,Results: 200 {'PUBCHEM.COMPOUND:2950270': 'True'},ARS Error: 200,Results: 200 {},Results: 200 {},Results: 200 {},ARS Error: 200,ARS Error: 200,ARS Error: 200,Results: 200 {},Results: 200 {},...,ARS Error: 200,ARS Error: 200,Error: 400,Error: 400,No Results: 200,"Results: 200 {'NCBIGene:120892': 'True', 'NCBI...","Results: 200 {'NCBIGene:3988': 'True', 'NCBIGe...",Results: 200 {},Results: 200 {},ARS Error: 200
ara-arax,No Results: 200,Results: 200 {},Results: 200 {},Results: 200 {},Results: 200 {},No Results: 200,Results: 200 {},Results: 200 {},Results: 200 {},Results: 200 {},...,Error: 598,Results: 200 {'MONDO:0005301': 'True'},Error: 598,Results: 200 {'CHEMBL.COMPOUND:CHEMBL1201607':...,Results: 200 {},Error: 598,"Results: 200 {'NCBIGene:3988': 'False', 'NCBIG...",Results: 200 {},Error: 598,Error:

In [26]:
df.replace('ARS Error', 'No Results', regex=True,inplace=True)

In [27]:
df

,A.0_RHOBTB2_direct,A.0_RHOBTB2_direct_inverse,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,...,B.3b_DILI-two-hop-drug,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule,C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
pk_id,https://arax.ncats.io/?source=ARS&id=90a3ab93-...,https://arax.ncats.io/?source=ARS&id=8b3142ae-...,https://arax.ncats.io/?source=ARS&id=e9ea07a4-...,https://arax.ncats.io/?source=ARS&id=9ad71701-...,https://arax.ncats.io/?source=ARS&id=69048de0-...,https://arax.ncats.io/?source=ARS&id=cf8a9ce5-...,https://arax.ncats.io/?source=ARS&id=566ba27c-...,https://arax.ncats.io/?source=ARS&id=51991eb6-...,https://arax.ncats.io/?source=ARS&id=6661979d-...,https://arax.ncats.io/?source=ARS&id=545a14e8-...,...,https://arax.ncats.io/?source=ARS&id=8e0c4d64-...,https://arax.ncats.io/?source=ARS&id=439a699a-...,https://arax.ncats.io/?source=ARS&id=21680308-...,https://arax.ncats.io/?source=ARS&id=20dddccd-...,https://arax.ncats.io/?source=ARS&id=8545ddfd-...,https://arax.ncats.io/?source=ARS&id=da789ff3-...,https://arax.ncats.io/?source=ARS&id=87a3de47-...,https://arax.ncats.io/?source=ARS&id=ad6f1db8-...,https://arax.ncats.io/?source=ARS&id=ac9c22cc-...,https://arax.ncats.io/?source=ARS&id=943660a4-...
ara-aragorn,No Results: 200,Results: 200 {},Results: 200 {},Error: 598,No Results: 200,Results: 200 {},Results: 200 {},No Results: 200,Results: 200 {},Results: 200 {},...,Error: 598,Results: 200 {'MONDO:0005301': 'True'},Error: 598,Results: 200 {'CHEMBL.COMPOUND:CHEMBL1201607':...,Results: 200 {},Error: 598,Error: 598,Results: 200 {},Results: 200 {},Error: 598
ara-aragorn-exp,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,...,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404
kp-cam,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-explanatory,Error: 500,Results: 200 {},Results: 200 {},No Results: 200,No Results: 200,Results: 200 {},No Results: 200,No Results: 200,Results: 200 {},No Results: 200,...,No Results: 200,Results: 200 {'MONDO:0005301': 'True'},No Results: 200,No Results: 200,Results: 200 {},Error: 504,No Results: 200,Results: 200 {},Results: 200 {},Error: 504
kp-icees,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,Error: 598,No Results: 200,No Results: 200,No Results: 200,Error: 598,No Results: 200,Error: 598,No Results: 200,Error: 598
ara-improving,Results: 200 {'PUBCHEM.COMPOUND:2950270': 'True'},No Results: 200,Results: 200 {},Results: 200 {},Results: 200 {},No Results: 200,No Results: 200,No Results: 200,Results: 200 {},Results: 200 {},...,No Results: 200,No Results: 200,Error: 400,Error: 400,No Results: 200,"Results: 200 {'NCBIGene:120892': 'True', 'NCBI...","Results: 200 {'NCBIGene:3988': 'True', 'NCBIGe...",Results: 200 {},Results: 200 {},No Results: 200
ara-arax,No Results: 200,Results: 200 {},Results: 200 {},Results: 200 {},Results: 200 {},No Results: 200,Results: 200 {},Results: 200 {},Results: 200 {},Results: 200 {},...,Error: 598,Results: 200 {'MONDO:0005301': 'True'},Error: 598,Results: 200 {'CHEMBL.COMPOUND:CHEMBL1201607':...,Results: 200 {},Error: 598,"Results: 200 {'NCBIGene:3988': 'False', 'NCBIG...",Results: 200 {}

In [28]:
df.replace('{}','',regex=True,inplace=True)

### Creating second table with edge attribute source

<br>

In [29]:
final_dict2 = defaultdict(dict)
for k in sorted(workflow_result_messages):
    print(k)
    col.append(k)
    
    count = 0
    
    for key, value in workflow_result_messages[k][1].items():
        final_dict2[k][key] = value

A.0_RHOBTB2_direct
A.0_RHOBTB2_direct_inverse
A.1_RHOBTB2
A.2_RHOBTB2_twohop
A.2_RHOBTB2_twohop_constrained
A.2a_RHOBTB2_twohop
A.2a_expanded_RHOBTB2_twohop
A.2a_expanded_RHOBTB2_twohop_constrained
A.3_KCNMA1
A.8_EGFR_simple
A.9_EGFR_advanced
A.Multiomics_BigGIM_DR_KP_EGFR
A.Multiomics_BigGIM_DR_KP_RHOBTB2
B.1_DILI-three-hop
B.2a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone
B.2b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol
B.2c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin
B.2d_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone
B.2e_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5865_Prednisone
B.3a_DILI-two-hop-chemical-entity
B.3b_DILI-two-hop-drug
C.1a_SmallMolecule_real_world_evidence_MultSclerosis
C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule
C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule
C.3a_MultSclerosis_related_to_Nimodipine
D.1_parkinsons-crohns
D.2_ssri-heart-disease
D.3_ssri-heart-disease-one-hop
D.4_tryptophan-kynurenine
D.6_metfo

In [30]:
final_dict2

defaultdict(dict,
            {'A.0_RHOBTB2_direct': {'ara-improving': {'infores:improving-agent',
               'infores:lincs',
               'infores:spoke'}},
             'A.0_RHOBTB2_direct_inverse': {'ara-aragorn': {'infores:aragorn',
               'infores:aragorn-ranker-ara',
               'infores:automat-covidkop',
               'infores:automat-ctd',
               'infores:biothings-multiomics-biggim-drugresponse',
               'infores:ctd',
               'infores:translator-biothings-explorer'},
              'ara-arax': {'infores:arax',
               'infores:automat-ctd',
               'infores:biothings-multiomics-biggim-drugresponse',
               'infores:ctd',
               'infores:translator-biothings-explorer'},
              'ara-bte': {'infores:automat-ctd',
               'infores:ctd',
               'infores:translator-biothings-explorer'},
              'ara-explanatory': {'infores:automat-ctd',
               'infores:automat-robokop',
      

In [31]:
final_dictassemble = []
for k, vs in final_dict2.items():
    #print(k,vs)
    for kv, v in vs.items():
        for t in v:
            final_dictassemble.append([k,kv,t])

In [32]:
final_dictassemble

[['A.0_RHOBTB2_direct', 'ara-improving', 'infores:improving-agent'],
 ['A.0_RHOBTB2_direct', 'ara-improving', 'infores:lincs'],
 ['A.0_RHOBTB2_direct', 'ara-improving', 'infores:spoke'],
 ['A.0_RHOBTB2_direct_inverse', 'ara-aragorn', 'infores:aragorn-ranker-ara'],
 ['A.0_RHOBTB2_direct_inverse', 'ara-aragorn', 'infores:automat-ctd'],
 ['A.0_RHOBTB2_direct_inverse', 'ara-aragorn', 'infores:aragorn'],
 ['A.0_RHOBTB2_direct_inverse', 'ara-aragorn', 'infores:ctd'],
 ['A.0_RHOBTB2_direct_inverse',
  'ara-aragorn',
  'infores:biothings-multiomics-biggim-drugresponse'],
 ['A.0_RHOBTB2_direct_inverse', 'ara-aragorn', 'infores:automat-covidkop'],
 ['A.0_RHOBTB2_direct_inverse',
  'ara-aragorn',
  'infores:translator-biothings-explorer'],
 ['A.0_RHOBTB2_direct_inverse', 'ara-arax', 'infores:automat-ctd'],
 ['A.0_RHOBTB2_direct_inverse', 'ara-arax', 'infores:arax'],
 ['A.0_RHOBTB2_direct_inverse', 'ara-arax', 'infores:ctd'],
 ['A.0_RHOBTB2_direct_inverse',
  'ara-arax',
  'infores:biothings-multi

In [33]:
column_names = ['Workflow', 'ARS-KPs', 'Values']
df2 = pd.DataFrame(final_dictassemble, columns=column_names)
df2 = df2.astype(str)

In [34]:
df2.Values = df2.Values.apply(lambda x: x[2:-2] if ('[' in x) else x)

In [35]:
df2test = df2.groupby(['Workflow','Values'])['ARS-KPs'].agg(list)

In [36]:
df2test = pd.DataFrame(df2test.unstack().T)

In [37]:
df2test

Workflow,A.0_RHOBTB2_direct,A.0_RHOBTB2_direct_inverse,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,...,B.2e_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5865_Prednisone,B.3a_DILI-two-hop-chemical-entity,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule,C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine
Values,,,,,,,,,,,,,,,,,,,,,
CTD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"[ara-bte, ara-aragorn, ara-arax]",NaN,[ara-bte],"[ara-arax, ara-bte]",NaN,NaN
DisGeNET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,[ara-arax],NaN,[ara-bte],"[ara-arax, ara-bte]",NaN,NaN
DrugCentral Indication,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"[ara-bte, ara-aragorn, ara-arax]",NaN,[ara-bte],"[ara-arax, ara-bte]",NaN,NaN
JensenLab Knowledge AmyCo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,[ara-bte],NaN,NaN,NaN
JensenLab Knowledge GHR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,[ara-bte],NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
infores:stitch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ara-arax],NaN,NaN
infores:tcrd,NaN,NaN,NaN,NaN,NaN,NaN,[ara-aragorn],NaN,NaN,"[ara-arax, kp-molecular, ara-unsecret, ara-ara...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
infores:text-mining-provider-targeted,NaN,NaN,"[ara-aragorn, kp-textmining, ara-arax, ara-bte]","[kp-textmining, ara-arax]","[ara-arax, kp-textmining, ara-bte]","[kp-textmining, ara-aragorn, ara-bte]","[ara-arax, ara-bte, ara-aragorn, kp-textmining]","[kp-textmining, ara-bte, ara-arax]","[kp-textmining, ara-arax, ara-bte, ara-aragorn]","[ara-arax, kp-textmining, ara-bte, ara-aragorn]",...,"[kp-textmining, ara-bte, ara-arax]",NaN,NaN,NaN,NaN,"[ara-aragorn, ara-arax, ara-bte]","[ara-bte, kp-textmining]","[ara-arax, ara-bte, kp-textmining]","[kp-textmining, ara-aragorn, ara-bte]",NaN


In [38]:
#df2test.drop([''], axis=0, inplace=True)

In [39]:
#df2test.index = df2test.index.map(lambda x: x[2:-2] if ('[' in x) else x)

In [40]:
df2test

Workflow,A.0_RHOBTB2_direct,A.0_RHOBTB2_direct_inverse,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,...,B.2e_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5865_Prednisone,B.3a_DILI-two-hop-chemical-entity,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule,C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine
Values,,,,,,,,,,,,,,,,,,,,,
CTD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"[ara-bte, ara-aragorn, ara-arax]",NaN,[ara-bte],"[ara-arax, ara-bte]",NaN,NaN
DisGeNET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,[ara-arax],NaN,[ara-bte],"[ara-arax, ara-bte]",NaN,NaN
DrugCentral Indication,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"[ara-bte, ara-aragorn, ara-arax]",NaN,[ara-bte],"[ara-arax, ara-bte]",NaN,NaN
JensenLab Knowledge AmyCo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,[ara-bte],NaN,NaN,NaN
JensenLab Knowledge GHR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,[ara-bte],NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
infores:stitch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ara-arax],NaN,NaN
infores:tcrd,NaN,NaN,NaN,NaN,NaN,NaN,[ara-aragorn],NaN,NaN,"[ara-arax, kp-molecular, ara-unsecret, ara-ara...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
infores:text-mining-provider-targeted,NaN,NaN,"[ara-aragorn, kp-textmining, ara-arax, ara-bte]","[kp-textmining, ara-arax]","[ara-arax, kp-textmining, ara-bte]","[kp-textmining, ara-aragorn, ara-bte]","[ara-arax, ara-bte, ara-aragorn, kp-textmining]","[kp-textmining, ara-bte, ara-arax]","[kp-textmining, ara-arax, ara-bte, ara-aragorn]","[ara-arax, kp-textmining, ara-bte, ara-aragorn]",...,"[kp-textmining, ara-bte, ara-arax]",NaN,NaN,NaN,NaN,"[ara-aragorn, ara-arax, ara-bte]","[ara-bte, kp-textmining]","[ara-arax, ara-bte, kp-textmining]","[kp-textmining, ara-aragorn, ara-bte]",NaN


In [41]:
#df2.replace([], 'None', regex=True,inplace=True)

In [42]:
#df2test = df2test.mask(df2test.applymap(type).eq(list) & ~df2test.astype(bool))

In [43]:
df2test = df2test.rename_axis(None)

In [44]:
df2test.columns.name = None

In [45]:
df2test.fillna('', inplace=True)

In [46]:
df2test

,A.0_RHOBTB2_direct,A.0_RHOBTB2_direct_inverse,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,...,B.2e_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5865_Prednisone,B.3a_DILI-two-hop-chemical-entity,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule,C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine
CTD,,,,,,,,,,,...,,,,,"[ara-bte, ara-aragorn, ara-arax]",,[ara-bte],"[ara-arax, ara-bte]",,
DisGeNET,,,,,,,,,,,...,,,,,[ara-arax],,[ara-bte],"[ara-arax, ara-bte]",,
DrugCentral Indication,,,,,,,,,,,...,,,,,"[ara-bte, ara-aragorn, ara-arax]",,[ara-bte],"[ara-arax, ara-bte]",,
JensenLab Knowledge AmyCo,,,,,,,,,,,...,,,,,,,[ara-bte],,,
JensenLab Knowledge GHR,,,,,,,,,,,...,,,,,,,[ara-bte],,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
infores:stitch,,,,,,,,,,,...,,,,,,,,[ara-arax],,
infores:tcrd,,,,,,,[ara-aragorn],,,"[ara-arax, kp-molecular, ara-unsecret, ara-ara...",...,,,,,,,,,,
infores:text-mining-provider-targeted,,,"[ara-aragorn, kp-textmining, ara-arax, ara-bte]","[kp-textmining, ara-arax]","[ara-arax, kp-textmining, ara-bte]","[kp-textmining, ara-aragorn, ara-bte]","[ara-arax, ara-bte, ara-aragorn, kp-textmining]","[kp-textmining, ara-bte, ara-arax]","[kp-textmining, ara-arax, ara-bte, ara-aragorn]","[ara-arax, kp-textmining, ara-bte, ara-aragorn]",...,"[kp-textmining, ara-bte, ara-arax]",,,,,"[ara-aragorn, ara-arax, ara-bte]","[ara-bte, kp-textmining]","[ara-arax, ara-bte, kp-textmining]","[kp-textmining, ara-aragorn, ara-bte]",
infores:translator-biothings-explorer,,"[ara-aragorn, ara-arax, ara-bte, ara-explanato...","[ara-aragorn, kp-textmining, ara-arax, ara-bte]","[kp-textmining, ara-arax]","[ara-arax, kp-textmining, ara-bte]","[kp-textmining, ara-aragorn, ara-bte]","[ara-arax, ara-bte, ara-aragorn, kp-textmining]","[kp-textmining, ara-bte, ara-arax]","[kp-textmining, ara-arax, ara-bte, ara-aragorn]","[ara-arax, kp-textmining, ara-bte, ara-aragorn]",...,"[kp-textmining, ara-bte, ara-arax]",[ara-bte],"[ara-aragorn, ara-arax, kp-textmining, ara-bte...",[ara-bte],"[ara-bte, ara-aragorn, ara-arax]","[ara-aragorn, ara-arax, ara-bte]","[ara-bte, kp-textmining]","[ara-arax, ara-bte, kp-textmining]","[ara-explanatory, kp-textmining, ara-aragorn, ...",


## Sorting The index to push infores ID's that are compliant to Column L and M in INFORES Catalog


<br>
<br>

In [47]:
infores_catalog = pd.read_csv("/Users/priyash/Documents/GitHub/minihackathons/Notebooks/InfoRes Catalog - Translator InfoRes Catalog.csv", header=1)

In [48]:
infores_catalog = infores_catalog[['id', 'name','translator category','has contributor']]

In [49]:
infores_catalog = infores_catalog[:335]

In [50]:
dict_map = {}
for i in df2test.index.values:
    if i in infores_catalog['id'].values:
        indices = infores_catalog[infores_catalog['id']==i].index[0]
        if pd.notnull(infores_catalog.iloc[indices]['has contributor']):
            dict_map[i] = infores_catalog.iloc[indices]['translator category']
        else:
            dict_map[i] = 'External Source'
    else:
        dict_map[i] = 'Illegal value'

In [51]:
dict_map

{'CTD': 'Illegal value',
 'DisGeNET': 'Illegal value',
 'DrugCentral Indication': 'Illegal value',
 'JensenLab Knowledge AmyCo': 'Illegal value',
 'JensenLab Knowledge GHR': 'Illegal value',
 'JensenLab Knowledge UniProtKB-KW': 'Illegal value',
 'JensenLab Text Mining': 'Illegal value',
 'eRAM': 'Illegal value',
 'icees-asthma': 'Illegal value',
 'icees-dili': 'Illegal value',
 'infores:aragorn': 'ARA',
 'infores:aragorn-ranker-ara': 'Illegal value',
 'infores:arax': 'ARA',
 'infores:automat-biolink': 'KP',
 'infores:automat-cord19': 'KP',
 'infores:automat-covidkop': 'KP',
 'infores:automat-ctd': 'KP',
 'infores:automat-drug-central': 'KP',
 'infores:automat-gtopdb': 'KP',
 'infores:automat-hetio': 'KP',
 'infores:automat-hmdb': 'KP',
 'infores:automat-ontology-hierarchy': 'KP',
 'infores:automat-pharos': 'KP',
 'infores:automat-robokop': 'KP',
 'infores:automat-uberongraph': 'KP',
 'infores:bindingdb': 'External Source',
 'infores:biolink-api': 'KP',
 'infores:biothings-dgidb': 'KP',

In [52]:
df2test['Translator_Category_Complaint_to_ColL&M_InforesCatalog']=df2test.index.map(dict_map)

In [53]:
df2test

,A.0_RHOBTB2_direct,A.0_RHOBTB2_direct_inverse,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,...,B.3a_DILI-two-hop-chemical-entity,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule,C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,Translator_Category_Complaint_to_ColL&M_InforesCatalog
CTD,,,,,,,,,,,...,,,,"[ara-bte, ara-aragorn, ara-arax]",,[ara-bte],"[ara-arax, ara-bte]",,,Illegal value
DisGeNET,,,,,,,,,,,...,,,,[ara-arax],,[ara-bte],"[ara-arax, ara-bte]",,,Illegal value
DrugCentral Indication,,,,,,,,,,,...,,,,"[ara-bte, ara-aragorn, ara-arax]",,[ara-bte],"[ara-arax, ara-bte]",,,Illegal value
JensenLab Knowledge AmyCo,,,,,,,,,,,...,,,,,,[ara-bte],,,,Illegal value
JensenLab Knowledge GHR,,,,,,,,,,,...,,,,,,[ara-bte],,,,Illegal value
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
infores:stitch,,,,,,,,,,,...,,,,,,,[ara-arax],,,External Source
infores:tcrd,,,,,,,[ara-aragorn],,,"[ara-arax, kp-molecular, ara-unsecret, ara-ara...",...,,,,,,,,,,External Source
infores:text-mining-provider-targeted,,,"[ara-aragorn, kp-textmining, ara-arax, ara-bte]","[kp-textmining, ara-arax]","[ara-arax, kp-textmining, ara-bte]","[kp-textmining, ara-aragorn, ara-bte]","[ara-arax, ara-bte, ara-aragorn, kp-textmining]","[kp-textmining, ara-bte, ara-arax]","[kp-textmining, ara-arax, ara-bte, ara-aragorn]","[ara-arax, kp-textmining, ara-bte, ara-aragorn]",...,,,,,"[ara-aragorn, ara-arax, ara-bte]","[ara-bte, kp-textmining]","[ara-arax, ara-bte, kp-textmining]","[kp-textmining, ara-aragorn, ara-bte]",,KP
infores:translator-biothings-explorer,,"[ara-aragorn, ara-arax, ara-bte, ara-explanato...","[ara-aragorn, kp-textmining, ara-arax, ara-bte]","[kp-textmining, ara-arax]","[ara-arax, kp-textmining, ara-bte]","[kp-textmining, ara-aragorn, ara-bte]","[ara-arax, ara-bte, ara-aragorn, kp-textmining]","[kp-textmining, ara-bte, ara-arax]","[kp-textmining, ara-arax, ara-bte, ara-aragorn]","[ara-arax, kp-textmining, ara-bte, ara-aragorn]",...,[ara-bte],"[ara-aragorn, ara-arax, kp-textmining, ara-bte...",[ara-bte],"[ara-bte, ara-aragorn, ara-arax]","[ara-aragorn, ara-arax, ara-bte]","[ara-bte, kp-textmining]","[ara-arax, ara-bte, kp-textmining]","[ara-explanatory, kp-textmining, ara-aragorn, ...",,ARA


In [54]:
df2test['sort']=pd.Categorical(df2test['Translator_Category_Complaint_to_ColL&M_InforesCatalog'], ["KP", "ARA",'External Source', 'Illegal Value'])
df2test =df2test.sort_values(['sort'])
df2test  = df2test.rename_axis(None)

In [55]:
df2test

,A.0_RHOBTB2_direct,A.0_RHOBTB2_direct_inverse,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,...,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule,C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,Translator_Category_Complaint_to_ColL&M_InforesCatalog,sort
infores:biothings-dgidb,,,,[ara-arax],"[kp-textmining, ara-bte]",,,,"[kp-textmining, ara-arax, ara-bte, ara-aragorn]",,...,,[ara-bte],"[ara-bte, ara-aragorn, ara-arax]",,,"[ara-arax, ara-bte]",,,KP,KP
infores:lincs,[ara-improving],,"[ara-aragorn, ara-improving]","[ara-improving, ara-arax]","[ara-explanatory, ara-improving, kp-textmining...",[ara-aragorn],[ara-aragorn],,"[ara-improving, ara-aragorn]",[ara-improving],...,,,,,,"[ara-arax, ara-bte, ara-improving]",,,KP,KP
infores:molepro,,,,[ara-arax],,,"[ara-arax, ara-aragorn]",,"[ara-arax, ara-unsecret, ara-aragorn, kp-molec...","[ara-arax, kp-molecular, ara-unsecret, ara-ara...",...,,,[ara-aragorn],,,[ara-arax],,,KP,KP
infores:text-mining-provider-targeted,,,"[ara-aragorn, kp-textmining, ara-arax, ara-bte]","[kp-textmining, ara-arax]","[ara-arax, kp-textmining, ara-bte]","[kp-textmining, ara-aragorn, ara-bte]","[ara-arax, ara-bte, ara-aragorn, kp-textmining]","[kp-textmining, ara-bte, ara-arax]","[kp-textmining, ara-arax, ara-bte, ara-aragorn]","[ara-arax, kp-textmining, ara-bte, ara-aragorn]",...,,,,"[ara-aragorn, ara-arax, ara-bte]","[ara-bte, kp-textmining]","[ara-arax, ara-bte, kp-textmining]","[kp-textmining, ara-aragorn, ara-bte]",,KP,KP
infores:mydisease-info,,,,,,,,,,,...,,[ara-bte],"[ara-bte, ara-aragorn, ara-arax]",,[ara-bte],"[ara-arax, ara-bte]","[ara-explanatory, ara-aragorn, ara-bte]",,KP,KP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
infores:faers,,,,,,,[ara-aragorn],,[ara-aragorn],[ara-aragorn],...,[ara-aragorn],,[ara-aragorn],,[ara-bte],,"[ara-aragorn, ara-bte]",,Illegal value,NaN
infores:hetio,,,[ara-aragorn],"[kp-textmining, ara-arax]","[ara-explanatory, ara-arax, kp-textmining, ara...",[ara-aragorn],[ara-aragorn],,"[kp-textmining, ara-arax, ara-bte, ara-aragorn]",[ara-aragorn],...,[ara-aragorn],[ara-bte],"[ara-bte, ara-aragorn, ara-arax]",,[ara-bte],"[ara-arax, ara-bte]","[ara-aragorn, ara-bte]",[ara-aragorn],Illegal value,NaN
infores:ontological-hierarchy,,,,,,[ara-aragorn],[ara-aragorn],,[ara-aragorn],[ara-aragorn],...,[ara-aragorn],,[ara-aragorn],,[ara-bte],,[ara-aragorn],,Illegal value,NaN
infores:pdsp,,,,,,,,,,,...,,,,,,[ara-arax],,,Illegal value,NaN


In [56]:
cols = df2test.columns.tolist()
cols = [cols[-2]] + cols[:-2]
df2test = df2test[cols]

In [57]:
df2test['Query Type'] = 'Sync'

df2test = df2test[['Query Type']+ list(df2test.columns[:-1])]

<br>

## Converting the Pk's to hyperlink

<br>

In [58]:
df.loc['pk_id'] = df.loc['pk_id'].apply(lambda x: make_hyperlink(x))

In [59]:
df.rename({'pk_id': 'pk'}, inplace=True)

In [60]:
df.sort_index(inplace=True)

In [61]:
df

,A.0_RHOBTB2_direct,A.0_RHOBTB2_direct_inverse,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,...,B.3b_DILI-two-hop-drug,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule,C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,No Results: 200,Results: 200,Results: 200,Error: 598,No Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,...,Error: 598,Results: 200 {'MONDO:0005301': 'True'},Error: 598,Results: 200 {'CHEMBL.COMPOUND:CHEMBL1201607':...,Results: 200,Error: 598,Error: 598,Results: 200,Results: 200,Error: 598
ara-aragorn-exp,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,...,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404
ara-arax,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,...,Error: 598,Results: 200 {'MONDO:0005301': 'True'},Error: 598,Results: 200 {'CHEMBL.COMPOUND:CHEMBL1201607':...,Results: 200,Error: 598,"Results: 200 {'NCBIGene:3988': 'False', 'NCBIG...",Results: 200,Error: 598,Error: 598
ara-bte,No Results: 200,Results: 200,Results: 200,Error: 500,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,...,No Results: 200,Results: 200 {'MONDO:0005301': 'True'},Results: 200 {'CHEBI:45783': 'False'},Results: 200 {'CHEMBL.COMPOUND:CHEMBL1201607':...,Results: 200,"Results: 200 {'NCBIGene:120892': 'True', 'NCBI...","Results: 200 {'NCBIGene:3988': 'True', 'NCBIGe...",Results: 200,No Results: 200,Error: 598
ara-explanatory,Error: 500,Results: 200,Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,...,No Results: 200,Results: 200 {'MONDO:0005301': 'True'},No Results: 200,No Results: 200,Results: 200,Error: 504,No Results: 200,Results: 200,Results: 200,Error: 504
ara-improving,Results: 200 {'PUBCHEM.COMPOUND:2950270': 'True'},No Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,...,No Results: 200,No Results: 200,Error: 400,Error: 400,No Results: 200,"Results: 200 {'NCBIGene:120892': 'True', 'NCBI...","Results: 200 {'NCBIGene:3988': 'True', 'NCBIGe...",Results: 200,Results: 200,No Results: 200
ara-ncats,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,...,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503
ara-robokop,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,...,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500
ara-unsecret,No Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,...,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 598,Error: 598,Error: 598,Results: 200,No Results: 200,No Results: 200
kp-cam,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200


<br>

### Highlight the cells In Excel

<br>

In [62]:
def highlight(v):
    if v.startswith('Results'):
        return 'background-color: %s' % 'green'
    elif v.startswith('Error'):
        return 'background-color: %s' % 'red'
    elif v.startswith('No Results'):
        return 'background-color: %s' % 'yellow'
    elif v.startswith('ARS Error'):
        return 'background-color: %s' % 'blue'
    elif v.startswith('Unknown'):
        return 'background-color: %s' % 'magneta'
    else:
        return

In [63]:
styled = df.style.applymap(highlight)

In [64]:
styled

,A.0_RHOBTB2_direct,A.0_RHOBTB2_direct_inverse,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,A.Multiomics_BigGIM_DR_KP_EGFR,A.Multiomics_BigGIM_DR_KP_RHOBTB2,B.1_DILI-three-hop,B.2a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone,B.2b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol,B.2c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin,B.2d_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone,B.2e_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5865_Prednisone,B.3a_DILI-two-hop-chemical-entity,B.3b_DILI-two-hop-drug,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule,C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,No Results: 200,Results: 200,Results: 200,Error: 598,No Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,No Results: 200,Error: 598,Results: 200,Results: 200,Results: 200,Results: 200,Error: 598,Error: 598,Results: 200 {'MONDO:0005301': 'True'},Error: 598,Results: 200 {'CHEMBL.COMPOUND:CHEMBL1201607': 'True'},Results: 200,Error: 598,Error: 598,Results: 200,Results: 200,Error: 598
ara-aragorn-exp,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404
ara-arax,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Error: 598,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Error: 598,Error: 598,Results: 200 {'MONDO:0005301': 'True'},Error: 598,Results: 200 {'CHEMBL.COMPOUND:CHEMBL1201607': 'False'},Results: 200,Error: 598,"Results: 200 {'NCBIGene:3988': 'False', 'NCBIGene:5627': 'False', 'NCBIGene:7043': 'False'}",Results: 200,Error: 598,Error: 598
ara-bte,No Results: 200,Results: 200,Results: 200,Error: 500,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Error: 598,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200 {'MONDO:0005301': 'True'},Results: 200 {'CHEBI:45783': 'False'},Results: 200 {'CHEMBL.COMPOUND:CHEMBL1201607': 'True'},Results: 200,"Results: 200 {'NCBIGene:120892': 'True', 'NCBIGene:11315': 'True', 'NCBIGene:110357': 'False'}","Results: 200 {'NCBIGene:3988': 'True', 'NCBIGene:5627': 'False', 'NCBIGene:7043': 'False'}",Results: 200,No Results: 200,Error: 598
ara-explanatory,Error: 500,Results: 200,Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200 {'MONDO:0005301': 'True'},No Results: 200,No Results: 200,Results: 200,Error: 504,No Results: 200,Results: 200,Results: 200,Error: 504
ara-improving,Results: 200 {'PUBCHEM.COMPOUND:2950270': 'True'},No Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 400,Error: 400,No Results: 200,"Results: 200 {'NCBIGene:120892': 'True', 'NCBIGene:11315': 'True', 'NCBIGene:110357': 'False'}","Result

In [65]:
date = datetime.now().strftime("%Y_%m_%d-%I_%M_%S_%p")
wks_name = 'Workflow Progress Tracker_' + date

In [66]:
styled.to_excel('/Users/priyash/Documents/GitHub/minihackathons/Notebooks/' + wks_name + '.xlsx')

#styled.to_excel(writer, sheet_name='progress_tracker')
#df2.to_excel(writer, sheet_name='edge_attribute')

/usr/local/lib/python3.9/site-packages/pandas/io/formats/excel.py:375: CSSWarning: Unhandled color format: 'magneta'
  warnings.warn(f"Unhandled color format: {repr(val)}", CSSWarning)


<br>

### Pushing dataframe to excel sheet on google drive

<br>


**Here I am using the google drive API to push the daatframe into an axcel sheet 
Every individula has the unique credential file that they need to create for google drive API -- 
"araxworkflowprogresstesting-2632632db8be.json" -- is the credential used from my drive. place this json file where
the ReadAndRunAllWorkFLows.ipynb will be. NB: i have removed my credntial file for privacy reasons. Always remove
the json file before making committs to the repo. To use googe Drive API follow: https://towardsdatascience.com/how-to-manage-files-in-google-drive-with-python-d26471d91ecd**

<br>

In [67]:
styled

,A.0_RHOBTB2_direct,A.0_RHOBTB2_direct_inverse,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,A.Multiomics_BigGIM_DR_KP_EGFR,A.Multiomics_BigGIM_DR_KP_RHOBTB2,B.1_DILI-three-hop,B.2a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone,B.2b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol,B.2c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin,B.2d_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone,B.2e_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5865_Prednisone,B.3a_DILI-two-hop-chemical-entity,B.3b_DILI-two-hop-drug,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule,C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,No Results: 200,Results: 200,Results: 200,Error: 598,No Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,No Results: 200,Error: 598,Results: 200,Results: 200,Results: 200,Results: 200,Error: 598,Error: 598,Results: 200 {'MONDO:0005301': 'True'},Error: 598,Results: 200 {'CHEMBL.COMPOUND:CHEMBL1201607': 'True'},Results: 200,Error: 598,Error: 598,Results: 200,Results: 200,Error: 598
ara-aragorn-exp,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404
ara-arax,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Error: 598,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Error: 598,Error: 598,Results: 200 {'MONDO:0005301': 'True'},Error: 598,Results: 200 {'CHEMBL.COMPOUND:CHEMBL1201607': 'False'},Results: 200,Error: 598,"Results: 200 {'NCBIGene:3988': 'False', 'NCBIGene:5627': 'False', 'NCBIGene:7043': 'False'}",Results: 200,Error: 598,Error: 598
ara-bte,No Results: 200,Results: 200,Results: 200,Error: 500,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Error: 598,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200 {'MONDO:0005301': 'True'},Results: 200 {'CHEBI:45783': 'False'},Results: 200 {'CHEMBL.COMPOUND:CHEMBL1201607': 'True'},Results: 200,"Results: 200 {'NCBIGene:120892': 'True', 'NCBIGene:11315': 'True', 'NCBIGene:110357': 'False'}","Results: 200 {'NCBIGene:3988': 'True', 'NCBIGene:5627': 'False', 'NCBIGene:7043': 'False'}",Results: 200,No Results: 200,Error: 598
ara-explanatory,Error: 500,Results: 200,Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200 {'MONDO:0005301': 'True'},No Results: 200,No Results: 200,Results: 200,Error: 504,No Results: 200,Results: 200,Results: 200,Error: 504
ara-improving,Results: 200 {'PUBCHEM.COMPOUND:2950270': 'True'},No Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 400,Error: 400,No Results: 200,"Results: 200 {'NCBIGene:120892': 'True', 'NCBIGene:11315': 'True', 'NCBIGene:110357': 'False'}","Result

In [68]:
df['Query Type'] = 'Sync'

df = df[['Query Type']+ list(df.columns[:-1])]

In [69]:
#df.reset_index(inplace=True)

In [70]:
#df.rename({'index': 'ARA/KP'}, axis=1, inplace=True)

<br>

## Google Drive API

**Push the dataframe to a google sheet via google drive API and then format the google spread sheet to add hyperlink to pk's and color the cells

<br>

### Push Dataframe 1

<br>

In [71]:
import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe
from gspread_formatting import *

In [72]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'araxworkflowprogresstesting-2632632db8be.json', scope)
gc = gspread.authorize(credentials)

In [73]:
spreadsheet_key = '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A'
#spreadsheet_key = '1sPpBIkxrHbQNiTm5oPs9-5KrjsyXcgaVAxknJj-u8pY'
#wks_name = 'Workflow Progress Tracker_' + date
d2g.upload(df, spreadsheet_key, wks_name, credentials=credentials, row_names=True)

<Worksheet 'Workflow Progress Tracker_2021_10_22-09_45_22_AM' id:85145690>

### Push Dataframe 2

<br>

In [74]:
wks2 = 'edge_attribute_source_' + date

In [75]:
spreadsheet_key = '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A'
#spreadsheet_key = '1sPpBIkxrHbQNiTm5oPs9-5KrjsyXcgaVAxknJj-u8pY'
#wks_name = 'Workflow Progress Tracker_' + date
d2g.upload(df2test, spreadsheet_key, wks2, credentials=credentials, row_names=True)

<Worksheet 'edge_attribute_source_2021_10_22-09_45_22_AM' id:1955791400>